# Requesting OOI Data in NetCDF Format
*Written by Sage Lichtenwalner, Rutgers University*

*Modified by Lori Garzio*

Purpose: demonstrate how to use the OOI M2M interface (also known as the OOI API) to request downloadable files for an instrument.

For this example, we will download data for the PAR sensor on Pioneer Glider 335 (CP05MOAS-GL335-05-PARADM000). More information about this instrument, along with a review of the OOI 1.0 datasets, are here: https://datareview.marine.rutgers.edu/instruments/report/CP05MOAS-GL335-05-PARADM000.

# Getting Started with the API

Individual instrument data stream files can be downloaded directly from the [OOI Data Portal](https://ooinet.oceanobservatories.org). However, if you want to download multiple instruments and streams, it is useful to know how to build data requests using the OOI API. In order to use the OOI API, you will first need to create an account on the OOI Data Portal. Once you have done that, you will need to grab your API username and token, which can be found on your profile page. Enter your username and token below.

For more information on the OOI M2M services, see the OOI website: https://oceanobservatories.org/ooi-m2m-interface/


In [1]:
USERNAME = 'YOUR API USERNAME'
TOKEN =  'YOUR API TOKEN'

In [2]:
# packages and functions needed to run the notebook
import requests
import datetime
import time
import functions.common as cf

To access the OOI API, we will use the [python requests library](http://docs.python-requests.org/en/master/), which is very easy to use.  Here is the basic command format.

> `r = requests.get(url, params=parameters, auth=('user', 'pass'))`

All we have to do is specify the URL we want to access, along with our login information.

## How to find Instrument Information needed to use the API?
Note, in order to use the OOI API, you will need to know the various OOI codes or IDs to make a request.  Many of these are available in the [OOI Data Portal](https://ooinet.oceanobservatories.org), but you may find the [Rutgers OOI SE 1.0 Data Review Portal](https://datareview.marine.rutgers.edu/) helpful. On each instrument report page there is an Info tab in the top right corner that gives you information about the instrument, data streams, and parameters.

For the instrument in this example, you will need the following to make the request to the M2M API.
* the 3 parts of the Reference Designator
* the stream name, and
* the data delivery method

To make a data request, we construct a URL using the elements above using the following format:

> /sensor/inv/{subsite}/{node}/{sensor}/{method}/{stream}

We can also specify a number of additional optionals using the **params** array. We can specify a start (**beginDT**) and ending date/time (**endDT**) for our request.

In [3]:
# Instrument Information
site = 'CP05MOAS'
node = 'GL335'
instrument = '05-PARADM000'
method = 'recovered_host'
stream = 'parad_m_glider_recovered'

SENSOR_BASE_URL = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/'

# Create the request URL
data_request_url ='/'.join((SENSOR_BASE_URL, site, node, instrument, method, stream))

# All of the following are optional
params = {
  'beginDT':'2014-10-06T00:00:00.000Z',
  'endDT':'2018-09-30T00:00:00.000Z',
  'format':'application/netcdf',
  'include_provenance':'true',
  'include_annotations':'true'
}

Send the request.  (Note, these lines are commented out, to prevent accidental resubmission when running through the entire notebook quickly.)

In [4]:
#r = requests.get(data_request_url, params=params, auth=(USERNAME, TOKEN))
#data = r.json()

The response will contain URLs and some other metadata about the request

In [5]:
data

{'requestUUID': '7167d11f-a8fa-48c9-a20a-7a7a42d50dc5',
 'outputURL': 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/lgarzio@marine.rutgers.edu/20190517T180206-CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered/catalog.html',
 'allURLs': ['https://opendap.oceanobservatories.org/thredds/catalog/ooi/lgarzio@marine.rutgers.edu/20190517T180206-CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered/catalog.html',
  'https://opendap.oceanobservatories.org/async_results/lgarzio@marine.rutgers.edu/20190517T180206-CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered'],
 'sizeCalculation': 2297073836,
 'timeCalculation': 482,
 'numberOfSubJobs': 1760}

## Which data URL should I use?

The **first** URL in the **allURLs** key points to the THREDDS server, which allows for programmatic data access without downloading the entire file.

In [6]:
print(data['allURLs'][0])

https://opendap.oceanobservatories.org/thredds/catalog/ooi/lgarzio@marine.rutgers.edu/20190517T180206-CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered/catalog.html


The **second** URL in the **allURLs** key provides a direct link to a web server which you can use to quickly download files if you don't want to go through THREDDS.

In [7]:
print(data['allURLs'][1])

https://opendap.oceanobservatories.org/async_results/lgarzio@marine.rutgers.edu/20190517T180206-CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered


## How can you check when a request is complete?

We can use the second URL to check if a status.txt file has been written to the location. If true, then the request has completed and all data have been delivered to the THREDDS server.

In [3]:
%%time
check_complete = data['allURLs'][1] + '/status.txt'
r = requests.get(check_complete)
while r.status_code != requests.codes.ok:
        print('Data request is still fulfilling. Trying again in 1 minute.')
        time.sleep(60)
        r = requests.get(check_complete)
print('Data request has fulfilled.')

Data request has fulfilled.
CPU times: user 24.4 ms, sys: 5.79 ms, total: 30.2 ms
Wall time: 42 ms


Once the request has completed, you can use the get_nc_urls function in the common functions folder included here to list all of the NetCDF data files that you downloaded (this function uses the same code above to make sure the request has completed before listing all of the files). Those links can be used to access, analyze, and plot the data.

In [4]:
datasets = cf.get_nc_urls([data['allURLs'][0]])
datasets

Data request has fulfilled.


['https://opendap.oceanobservatories.org/thredds/dodsC/ooi/lgarzio@marine.rutgers.edu/20190517T180206-CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered/deployment0005_CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered_20170116T150223.595370-20170304T093047.153350.nc',
 'https://opendap.oceanobservatories.org/thredds/dodsC/ooi/lgarzio@marine.rutgers.edu/20190517T180206-CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered/deployment0005_CP05MOAS-GL335-03-CTDGVM000-recovered_host-ctdgv_m_glider_instrument_recovered_18990907T061321-20170304T093047.153350.nc',
 'https://opendap.oceanobservatories.org/thredds/dodsC/ooi/lgarzio@marine.rutgers.edu/20190517T180206-CP05MOAS-GL335-05-PARADM000-recovered_host-parad_m_glider_recovered/deployment0005_CP05MOAS-GL335-00-ENG000000-recovered_host-glider_gps_position_20170116T150227.215330-20170304T073207.688540.nc',
 'https://opendap.oceanobservatories.org/thredds/dodsC/ooi/lgarzio@marine.rutgers.edu/201

You can use this information to write your own tools for downloading multiple instruments at once, or use the interactive code for downloading data here: https://github.com/ooi-data-lab/data-download